## 

In [10]:
import gradio as gr

iface = gr.Interface.load("huggingface/Helsinki-NLP/opus-mt-en-zh",
                          examples=[["That is a nice word, posstive word."]]
                          )

iface.launch()


AssertionError: parameter: src must be one of dict_keys(['huggingface', 'models', 'spaces'])